### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [2]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_wesad as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.Phases.PHASE_ORDER


c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
importlib.reload(train)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    # train.Metrics.MEAN_SCL, 
    # train.Metrics.SCR_RATE, 
]
# ] + train.Metrics.STATISTICAL

In [5]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)

model_phases = dr.Phases.PHASE_ORDER

label_type = "stai"
threshold = "fixed"

x, y = train.Train_WESAD.get_wesad_data(
    metrics, model_phases, verbose=False, label_type=label_type, threshold=threshold, 
    normalize=True, combine_phases=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATAURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.88107828 0.90812505 0.90449497 0.90468732 0.90988425 0.87952018
 0.90458671 0.90721665 0.90803251 0.90415366 0.86459932 0.89111214
 0.90071123 0.8995206  0.90137015 0.86159447 0.88727579 0.88857205
 0.89217532 0.89218673 0.86318375 0.88879632 0.88988362 0.89270159
 0.89084395        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 15], Predictions: [0 1], [64  5]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 15], Predictions: [0 1], [66  3]
Training RF ...
Model RF, Actual: [0 1], [54 15], Predictions: [0 1], [67  2]
Training XGB ...
Model XGB, Actual: [0 1], [54 15], Predictions: [0 1], [65  4]

None


[('bpm', 229), ('rmssd', 189), ('sdnn', 162), ('hf_rr', 140), ('lf_rr', 87)]


[('bpm', 0.4920086535858726), ('rmssd', 0.18531870707773915), ('sdnn', 0.12936660137021158), ('hf_rr', 0.11872419061926

One or more of the test scores are non-finite: [0.880772   0.88318949 0.90122262 0.90069258 0.90077666 0.87268989
 0.8823738  0.88362516 0.88589645 0.88682791 0.86659276 0.87794728
 0.88573809 0.88328273 0.88308496 0.88506403 0.88873835 0.88921859
 0.88641216 0.88495122 0.8796177  0.87718519 0.88838058 0.88460692
 0.88589942        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 3, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [62  8], Predictions: [0 1], [54 16]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [62  8], Predictions: [0 1], [51 19]
Training RF ...
Model RF, Actual: [0 1], [62  8], Predictions: [0 1], [50 20]
Training XGB ...
Model XGB, Actual: [0 1], [62  8], Predictions: [0 1], [51 19]

None


[('rmssd', 228), ('bpm', 219), ('sdnn', 139), ('lf_rr', 119), ('hf_rr', 94)]


[('bpm', 0.47050907393577646), ('rmssd', 0.22372785825721936), ('sdnn', 0.13387476951930002), ('hf_rr', 0.101032903100

...

Grid search for XGB ...
SVM: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 6, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [54 16], Predictions: [0 1], [57 13]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [54 16], Predictions: [0 1], [49 21]
Training RF ...
Model RF, Actual: [0 1], [54 16], Predictions: [0 1], [48 22]
Training XGB ...
Model XGB, Actual: [0 1], [54 16], Predictions: [0 1], [46 24]

None


[('bpm', 240), ('rmssd', 165), ('sdnn', 153), ('hf_rr', 118), ('lf_rr', 106)]


[('bpm', 0.4905265512585752), ('rmssd', 0.17429760439542993), ('hf_rr', 0.14338245372074465), ('sdnn', 0.120554069817542

One or more of the test scores are non-finite: [0.847957   0.86366306 0.87548824 0.87320083 0.87245235 0.85818384
 0.86887579 0.88180588 0.88386967 0.88055501 0.8540271  0.86000983
 0.88658267 0.88412575 0.88316706 0.84236085 0.86850167 0.89071914
 0.88683824 0.88629253 0.84705493 0.86960864 0.89433607 0.88231811
 0.88394202        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 7, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [61  8], Predictions: [0 1], [63  6]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [61  8], Predictions: [0 1], [61  8]
Training RF ...
Model RF, Actual: [0 1], [61  8], Predictions: [0 1], [62  7]
Training XGB ...
Model XGB, Actual: [0 1], [61  8], Predictions: [0 1], [63  6]

None


[('bpm', 224), ('rmssd', 173), ('sdnn', 163), ('hf_rr', 118), ('lf_rr', 109)]


[('bpm', 0.4310644377769634), ('rmssd', 0.22394553613884313), ('sdnn', 0.14046366783937791), ('hf_rr', 0.12594348873877

One or more of the test scores are non-finite: [0.8479349  0.8833047  0.88847789 0.90639003 0.9115084  0.86513179
 0.89486008 0.90245281 0.91069131 0.90893064 0.88392434 0.89712431
 0.90469215 0.91204545 0.91341466 0.86240363 0.88609735 0.90326422
 0.90743455 0.91093201 0.87803399 0.89125344 0.91032993 0.91116607
 0.9132418         nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [62  8], Predictions: [0 1], [65  5]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [62  8], Predictions: [0 1], [60 10]
Training RF ...
Model RF, Actual: [0 1], [62  8], Predictions: [0 1], [60 10]
Training XGB ...
Model XGB, Actual: [0 1], [62  8], Predictions: [0 1], [56 14]

None


[('bpm', 159), ('rmssd', 118), ('sdnn', 90), ('hf_rr', 73), ('lf_rr', 66)]


[('bpm', 0.47825301210584786), ('rmssd', 0.18483980741423275), ('hf_rr', 0.12861568662613887), ('sdnn', 0.12848183385858